In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.logging.set_verbosity(tf.logging.ERROR)

from sklearn.compose import make_column_transformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.isotonic import IsotonicRegression as IR

from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_score, recall_score, f1_score, \
                            average_precision_score, accuracy_score, \
                            balanced_accuracy_score, roc_auc_score, \
                            matthews_corrcoef
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay, PrecisionRecallDisplay, roc_curve, auc

import warnings

# Suppress all warnings (not recommended for production)
warnings.filterwarnings("ignore")

# Suppress specific warnings by category (more targeted)
warnings.filterwarnings("ignore", category=UserWarning)  # Example for UserWarning category

plt.rcParams['font.size'] = 20

%cd /raid/home/tanmoysarkar/Trustworthiness/Diabetes - Paper/Gender Disparity in Diabetes/ML4H 23 (Submission files)/Code

/raid/home/tanmoysarkar/Trustworthiness/Diabetes - Paper/Gender Disparity in Diabetes/ML4H 23 (Submission files)/Code


In [4]:
brfss_2021_mapped_cat = pd.read_csv('../../BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2021.csv')

brfss_2021_mapped_cat['Race'] = brfss_2021_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2019_mapped_cat = pd.read_csv('../../BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2019.csv')

brfss_2019_mapped_cat['Race'] = brfss_2019_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2017_mapped_cat = pd.read_csv('../../BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2017.csv')

brfss_2017_mapped_cat['Race'] = brfss_2017_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

brfss_2015_mapped_cat = pd.read_csv('../../BRFSS Datasets - Cleaned/Mapped categories (Step 2)/BRFSS 2015.csv')

brfss_2015_mapped_cat['Race'] = brfss_2015_mapped_cat['Race'].replace({1: 'White', 2: 'Black', 3: 'American Indian',
                                                               4: 'Asian', 5: 'Native Hawaiian', 6: 'Other race',
                                                               7: 'Multiracial', 8: 'Hispanic'})

print(brfss_2021_mapped_cat.shape, brfss_2019_mapped_cat.shape, brfss_2017_mapped_cat.shape, brfss_2015_mapped_cat.shape)

brfss_2021_one_hot = pd.read_csv('../../BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2021.csv')
brfss_2019_one_hot = pd.read_csv('../../BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2019.csv')
brfss_2017_one_hot = pd.read_csv('../../BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2017.csv')
brfss_2015_one_hot = pd.read_csv('../../BRFSS Datasets - Cleaned/One hot encoded (Step 3)/BRFSS 2015.csv')

print(brfss_2021_one_hot.shape, brfss_2019_one_hot.shape, brfss_2017_one_hot.shape, brfss_2015_one_hot.shape)

brfss_2021_one_hot['Race'] = brfss_2021_mapped_cat['Race'] # this is added to test performance over race
brfss_2019_one_hot['Race'] = brfss_2019_mapped_cat['Race'] # this is added to test performance over race
brfss_2017_one_hot['Race'] = brfss_2017_mapped_cat['Race'] # this is added to test performance over race
brfss_2015_one_hot['Race'] = brfss_2015_mapped_cat['Race'] # this is added to test performance over race

print(brfss_2021_one_hot.shape, brfss_2019_one_hot.shape, brfss_2017_one_hot.shape, brfss_2015_one_hot.shape)

(200136, 30) (210874, 30) (238130, 30) (220390, 30)
(200136, 49) (210874, 49) (238130, 49) (220390, 49)
(200136, 50) (210874, 50) (238130, 50) (220390, 50)


## Performance Metrics, Calibration, Threshold Selection

In [44]:
def performace_metrix(y_true, y_pred, y_pred_proba, group = "whole"):
  # Assuming you have true labels y_true and predicted labels y_pred
  # For class 0 and class 1

  #y_true, y_pred = y_test_2021, pred_test_2021_lr

  # Calculate precision, recall, and f1-score for each class
  precision_0 = precision_score(y_true, y_pred, pos_label=0)
  precision_1 = precision_score(y_true, y_pred, pos_label=1)

  recall_0 = recall_score(y_true, y_pred, pos_label=0)
  recall_1 = recall_score(y_true, y_pred, pos_label=1)

  f1_0 = f1_score(y_true, y_pred, pos_label=0)
  f1_1 = f1_score(y_true, y_pred, pos_label=1)

  # Calculate Area Under the Precision-Recall Curve for each class
  ap_0 = average_precision_score(y_true, y_pred_proba, pos_label=0)
  ap_1 = average_precision_score(y_true, y_pred_proba, pos_label=1)

  # Calculate overall accuracy
  accuracy = accuracy_score(y_true, y_pred)

  # Calculate balanced accuracy
  balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

  # Calculate Area Under the ROC Curve
  roc_auc = roc_auc_score(y_true, y_pred_proba)

  # Calculate Matthews Correlation Coefficient (MCC)
  mcc = matthews_corrcoef(y_true, y_pred)

  # Print the calculated metrics
  print("Class 1 Recall:", recall_1)
  print("Class 1 Precision:", precision_1)
  print("Class 1 Area Under Precision-Recall Curve:", ap_1)
  print("Class 1 F1-Score:", f1_1)


  print("Class 0 Recall:", recall_0)
  print("Class 0 Precision:", precision_0)
  print("Class 0 Area Under Precision-Recall Curve:", ap_0)
  print("Class 0 F1-Score:", f1_0)

  print("Overall Accuracy:", accuracy)
  print("Balanced Accuracy:", balanced_accuracy)
  print("Area Under ROC Curve:", roc_auc)
  print("Matthews Correlation Coefficient:", mcc)


  # Print the calculated metrics
  print(recall_1)
  print(precision_1)
  print(ap_1)
  print(f1_1)

  print(recall_0)
  print(precision_0)
  print(ap_0)
  print(f1_0)

  print(accuracy)
  print(balanced_accuracy)
  print(roc_auc)
  print(mcc)


  performance = [group, recall_1, precision_1, ap_1, f1_1, recall_0, precision_0, ap_0, f1_0, accuracy, balanced_accuracy, roc_auc, mcc]
  return performance

def insert_performance_dict(performance_dict, performance):
    c = 0
    for k in performance_dict.keys():
        performance_dict[k].append(performance[c])
        c+=1
    return performance_dict


from sklearn.isotonic import IsotonicRegression as IR

def isotonic(preds, labels, test_preds):
    preds = np.array(preds)
    labels = np.array(labels)
    test_preds = np.array(test_preds)

    ir = IR(out_of_bounds='clip')
    ir.fit( preds, labels )

    p_calibrated_v = ir.transform( preds )
    p_calibrated_t = ir.transform( test_preds )   # or ir.fit( p_test ), that's the same thing

    return p_calibrated_v, p_calibrated_t


import heapq
from sklearn.metrics import f1_score
from sklearn.metrics import balanced_accuracy_score

def Select_Threshold(df):
    full_threshold_list = []
    for threshold in np.arange(0,1.05,0.01):
        #df.drop(columns = ['y_pred'])
        df['y_pred'] = df['calibrated_prediction_prob'].apply(lambda x: 1 if x >= threshold else 0)
        
        y_pred = df["y_pred"].values
        y_true = df["y_true"].values
        
        f1_C1 = f1_score(y_true, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

        
        full_threshold_list.append([threshold, f1_C1, balanced_accuracy])
        
    df_varying_threshold = pd.DataFrame(full_threshold_list, columns = ['threshold', 'f1_score', 'balanced_accuracy'])
    
    # select three highest F1 score and the the highest balanced accuracy
    f1_scores = df_varying_threshold["f1_score"].values
    thresholds = df_varying_threshold["threshold"].values
    bal_acc_values = list(df_varying_threshold["balanced_accuracy"].values)
    
    #print(heapq.nlargest(3, f1_scores))
    list_index = heapq.nlargest(3, range(len(f1_scores)), key=f1_scores.__getitem__)
    opt_threshold = thresholds[bal_acc_values.index(max(bal_acc_values[list_index[0]], bal_acc_values[list_index[1]], bal_acc_values[list_index[2]]))]
    
    
    return opt_threshold, df_varying_threshold

def pred_label_using_threshold(y_pred_p, th = 0.195):
    y_pred_label = []
    for y in y_pred_p:
        if y <= th:
            y_pred_label.append(0)
        else:
            y_pred_label.append(1)
    return np.array(y_pred_label)


## Creating train, test, and validation set for entire experimental set up

In [17]:
random_state = 1

df_train_2021, df_test_2021 = train_test_split(brfss_2021_one_hot, test_size=0.4, random_state=random_state)
df_test_2021, df_validation_2021 = train_test_split(df_test_2021, test_size=0.5, random_state=random_state)
print(brfss_2021_one_hot.shape, df_train_2021.shape, df_test_2021.shape, df_validation_2021.shape)

X_train_2021 = df_train_2021.drop(['Diabetes', 'Race'],axis=1)
y_train_2021 = df_train_2021['Diabetes']
y_train_2021 = y_train_2021.to_numpy()

X_test_2021 = df_test_2021.drop(['Diabetes', 'Race'],axis=1)
y_test_2021 = df_test_2021['Diabetes']
y_test_2021 = y_test_2021.to_numpy()

X_validation_2021 = df_validation_2021.drop(['Diabetes', 'Race'],axis=1)
y_validation_2021 = df_validation_2021['Diabetes']
y_validation_2021 = y_validation_2021.to_numpy()

print(X_train_2021.shape, X_test_2021.shape, X_validation_2021.shape)



(200136, 50) (120081, 50) (40027, 50) (40028, 50)
(120081, 48) (40027, 48) (40028, 48)


### Whole group threshold calculation

In [33]:
sum_ = 0

for i in range(5):

    random_state = i

    df_train_2021, df_test_2021 = train_test_split(brfss_2021_one_hot, test_size=0.4, random_state=random_state)
    df_test_2021, df_validation_2021 = train_test_split(df_test_2021, test_size=0.5, random_state=random_state)

    X_train_2021 = df_train_2021.drop(['Diabetes', 'Race'],axis=1)
    y_train_2021 = df_train_2021['Diabetes']
    y_train_2021 = y_train_2021.to_numpy()

    X_test_2021 = df_test_2021.drop(['Diabetes', 'Race'],axis=1)
    y_test_2021 = df_test_2021['Diabetes']
    y_test_2021 = y_test_2021.to_numpy()

    X_validation_2021 = df_validation_2021.drop(['Diabetes', 'Race'],axis=1)
    y_validation_2021 = df_validation_2021['Diabetes']
    y_validation_2021 = y_validation_2021.to_numpy()

    lr = LogisticRegression(solver='liblinear')
    lr.fit(X_train_2021, y_train_2021)

    pred_prob_test_lr = lr.predict_proba(X_test_2021)[:, 1]
    pred_prob_validation_lr = lr.predict_proba(X_validation_2021)[:, 1]

    p_calibrated_val, p_calibrated_test = isotonic(pred_prob_validation_lr, y_validation_2021, pred_prob_test_lr)

    X_validation_2021_preds = X_validation_2021.copy()
    X_validation_2021_preds['y_true'] = y_validation_2021
    X_validation_2021_preds['calibrated_prediction_prob'] = p_calibrated_val

    th, th_df = Select_Threshold(X_validation_2021_preds)
    #th_df.to_csv("Threshold-selection/threshold_moving_trial_"+str(random_state)+".csv", index = False)
    print(i, th)
    sum_ += th


print(sum_ / (i+1))

0 0.24
1 0.24
2 0.24
3 0.22
4 0.24
0.236


### Subgroup threshold calculation

In [36]:
from collections import defaultdict
th_dict = defaultdict(list)

for state in range(5):
    print("============================ ", state , "============================ ")
    random_state = state

    df_train_2021, df_test_2021 = train_test_split(brfss_2021_one_hot, test_size=0.4, random_state=random_state)
    df_test_2021, df_validation_2021 = train_test_split(df_test_2021, test_size=0.5, random_state=random_state)

    X_train_2021 = df_train_2021.drop(['Diabetes', 'Race'],axis=1)
    y_train_2021 = df_train_2021['Diabetes']
    y_train_2021 = y_train_2021.to_numpy()

    X_test_2021 = df_test_2021.drop(['Diabetes', 'Race'],axis=1)
    y_test_2021 = df_test_2021['Diabetes']
    y_test_2021 = y_test_2021.to_numpy()

    X_validation_2021 = df_validation_2021.drop(['Diabetes', 'Race'],axis=1)
    y_validation_2021 = df_validation_2021['Diabetes']
    y_validation_2021 = y_validation_2021.to_numpy()

    lr = LogisticRegression(solver='liblinear')
    lr.fit(X_train_2021, y_train_2021)

    pred_prob_test_lr = lr.predict_proba(X_test_2021)[:, 1]
    pred_prob_validation_lr = lr.predict_proba(X_validation_2021)[:, 1]

    p_calibrated_val, p_calibrated_test = isotonic(pred_prob_validation_lr, y_validation_2021, pred_prob_test_lr)

    X_validation_2021_preds = X_validation_2021.copy()
    X_validation_2021_preds['y_true'] = y_validation_2021
    X_validation_2021_preds['calibrated_prediction_prob'] = p_calibrated_val


    age_group = [3, 4, 5]

    for i in age_group:
        
        X_validation_2021_age_group = X_validation_2021_preds[X_validation_2021_preds.Age == i]
        th, th_df = Select_Threshold(X_validation_2021_age_group)
        #th_df.to_csv("Threshold-selection/threshold_moving_trial_"+str(random_state)+".csv", index = False)
        print(i, th)
        th_dict[i].append(th)

============================  0 ============================ 
3 0.17
4 0.21
5 0.21
============================  1 ============================ 
3 0.18
4 0.21
5 0.24
============================  2 ============================ 
3 0.14
4 0.24
5 0.18
============================  3 ============================ 
3 0.12
4 0.18
5 0.18
============================  4 ============================ 
3 0.13
4 0.2
5 0.2


In [40]:
for k in th_dict.keys():
    print(k, sum(th_dict[k]) / len(th_dict[k]))

3 0.148
4 0.20799999999999996
5 0.20199999999999996


In [45]:
save_dir = "Performance-LR/"
state = "subgrup_threshold"

random_state = 1

df_train_2021, df_test_2021 = train_test_split(brfss_2021_one_hot, test_size=0.4, random_state=random_state)
df_test_2021, df_validation_2021 = train_test_split(df_test_2021, test_size=0.5, random_state=random_state)

X_train_2021 = df_train_2021.drop(['Diabetes', 'Race'],axis=1)
y_train_2021 = df_train_2021['Diabetes']
y_train_2021 = y_train_2021.to_numpy()

X_test_2021 = df_test_2021.drop(['Diabetes', 'Race'],axis=1)
y_test_2021 = df_test_2021['Diabetes']
y_test_2021 = y_test_2021.to_numpy()

X_validation_2021 = df_validation_2021.drop(['Diabetes', 'Race'],axis=1)
y_validation_2021 = df_validation_2021['Diabetes']
y_validation_2021 = y_validation_2021.to_numpy()


lr = LogisticRegression(solver='liblinear')
lr.fit(X_train_2021, y_train_2021)

pred_prob_test_lr = lr.predict_proba(X_test_2021)[:, 1]
pred_prob_validation_lr = lr.predict_proba(X_validation_2021)[:, 1]

# calibration
ir = IR(out_of_bounds='clip')
ir.fit(pred_prob_validation_lr, y_validation_2021 )
pred_prob_validation_lr_calibrated = ir.transform( pred_prob_validation_lr )
pred_prob_test_lr_calibrated = ir.transform( pred_prob_test_lr )


X_test_2021_preds = X_test_2021.copy()
X_test_2021_preds['y_true'] = y_test_2021
X_test_2021_preds['calibrated_prediction_prob'] = pred_prob_test_lr_calibrated

performance_dict = {
        "Group ": [],
        "Class 1 Recall": [],
        "Class 1 Precision": [],
        "Class 1 AU_PRC": [],
        "Class 1 F1-Score": [],
        "Class 0 Recall": [],
        "Class 0 Precision": [],
        "Class 0 AU_PRC": [],
        "Class 0 F1-Score": [],
        "Overall Accuracy": [],
        "Balanced Accuracy": [],
        "AU_ROC": [],
        "MCC": []
    }

age_group_th = {
    3: 0.148,
    4: 0.21,
    5: 0.20
}

for i in age_group_th.keys():
    test_group = X_test_2021_preds[X_test_2021_preds.Age == i]
    print("============================ ", i , "============================ ")

    X_test_group = X_test_2021_preds.drop(['y_true', 'calibrated_prediction_prob'],axis=1)

    y_test_group = test_group['y_true']
    y_test_group = y_test_group.to_numpy()

    y_test_pred_calibrated_group = test_group['calibrated_prediction_prob']
    y_test_pred_calibrated_group = y_test_pred_calibrated_group.to_numpy()

    # thresholding
    pred_test_group = pred_label_using_threshold(y_test_pred_calibrated_group, age_group_th[i])

    performace = performace_metrix(y_test_group, pred_test_group, y_test_pred_calibrated_group, i)
    performance_dict = insert_performance_dict(performance_dict, performace)

all_performances = pd.DataFrame(performance_dict)
all_performances.to_csv(save_dir + state +  ".csv", index=False)


============================  3 ============================ 
Class 1 Recall: 0.45614035087719296
Class 1 Precision: 0.15757575757575756
Class 1 Area Under Precision-Recall Curve: 0.15722124531485454
Class 1 F1-Score: 0.2342342342342342
Class 0 Recall: 0.9352283317800559
Class 0 Precision: 0.9847890088321885
Class 0 Area Under Precision-Recall Curve: 0.9378767316237041
Class 0 F1-Score: 0.9593690248565966
Overall Accuracy: 0.9228325011348162
Balanced Accuracy: 0.6956843413286244
Area Under ROC Curve: 0.8441776622357384
Matthews Correlation Coefficient: 0.2360447226393439
0.45614035087719296
0.15757575757575756
0.15722124531485454
0.2342342342342342
0.9352283317800559
0.9847890088321885
0.9378767316237041
0.9593690248565966
0.9228325011348162
0.6956843413286244
0.8441776622357384
0.2360447226393439
============================  4 ============================ 
Class 1 Recall: 0.4251968503937008
Class 1 Precision: 0.2857142857142857
Class 1 Area Under Precision-Recall Curve: 0.26446740895